In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
# Load some envs since we are using a private bucket for now
from dotenv import load_dotenv
load_dotenv('.env')
from fashion_clip.fashion_clip import FCLIPDataset, FashionCLIP
from fashion_clip.utils import display_images_from_url
from app.gradient_rec import GradREC

In [ ]:
# load a FashionCLIP modfel
dataset = FCLIPDataset(name='FF',
                       image_source_type='s3',
                       image_source_path ='s3://farfetch-images-ztapq86olwi6kub2p79d/images/')
fclip = FashionCLIP('FCLIP', dataset)

In [ ]:
# instantiate GradREC object
gradrec =  GradREC(fclip)

In [ ]:
# Supply a start and end query to obtain semantic direction of traversal
start_query = 'long skirt'
end_query = 'short skirt'
v_dir = gradrec.direction_vector(start_query=start_query, end_query=end_query)

In [ ]:
# Use FashionCLIP retrieval to get some initial seed products
start_points = fclip.retrieval([start_query])

In [ ]:
path = gradrec.traverse_space(start_point=fclip.image_vectors[start_points[0][1]],
                              search_space=fclip.image_vectors,
                              v_dir=v_dir,
                              step_size=-2.0,
                              steps=20,
                              reg_space=fclip.image_vectors,
                              reg_weight=0.9)
# convert path from index to id
path = [[dataset.ids[idx] for idx in p] for p in path]

In [ ]:
for p in path:
    dataset.display_products(p, fields=tuple(), columns=10)

In [ ]:
print(path)
